<a href="https://colab.research.google.com/github/DataStrategisthjk/NLP/blob/main/DNN_sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
                                       # header: what to save in the column
                                       # delimiter
                                       # quoting
train = pd.read_csv("ratings_train.txt", header=0, delimiter="\t", quoting=3)
train

#### 네이버 영화 리뷰 데이터로 감성분류를 할건데,
#### 간단한 전처리를 해줘야 딥러닝 모델에 들어갈 수 있다.

In [ ]:
!pip install konlpy

In [ ]:
import re
from konlpy.tag import Okt #konlpy.tag okt are libraries for korean language.

okt = Okt()

text = "안녕하세요 오승환 과외 선생입니다."

okt.morphs(text, stem=True)

In [ ]:
okt.morphs(text, stem=False)

1. 문자열을 추출
2. 정규표현식으로 필터링. (특수문자, 이모티콘)
3. stopword 제거해서 리스트를 만들었다.

In [ ]:
stop_word = ['은', '는','이', '가','이다']

def preprocessing(content, okt):
    content_re = re.sub("[^가-힣 ]", "",content) # 정규표현식으로 필터링
    content_word = okt.morphs(content_re, stem=True)
    #정규표현식으로 전처리된 문장을 단어의 어간 단위로 분리

    word_list = [] # 스톱워드 해당되지 않는 워드들을 담을 그릇

    for word in content_word: # content_word에서 한글자씩 뽑아냄
        if word not in stop_word: # 뽑아낸 워드가 스톱워드에 속하지 않는다면,
            word_list.append(word) # word list에 담아낸다.

    return word_list # 정규표현식으로 필터링하고, 단어어간 분리하고, 스톱워드 제거한,
                     # 리뷰 한문장을 리턴한다. 예: ["이", "영화", "재밌다."]

In [ ]:
preprocessing("안녕하세요 오승환입니다. 감성분류를 하고 있습니다.", okt)


In [ ]:
# 네이버 영화리뷰 데이터의 모든 리뷰들을 preprocessing 해줘야한다.
# 아래의 코드로 모두 preprocessing 해보자.

train_review = [] # 전처리된 문장들을 담을 빈 리스트

for review in train['document'][:500]: # 15만개 다 할 수 없으니 500개만~!
    train_review.append(preprocessing(review, okt)) #preprocessing 함수에 리뷰와 단어어간분리
                                                    # 라이브러리를 넣어서, 리턴 받은 결과를
                                                    # train review에 차곡 차곡 쌓는다
                                                    # 그러면 train review는 2차원 배열이 된다.


In [ ]:
train_review

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer() # 단어를 숫자로 변환시켜주는 도구

# 토크나이저로 전체 순서를 정의 #단어별로 숫자를 정의한다. #단어 숫자 사전을 만든다
tokenizer.fit_on_texts(train_review)

# 토크나이저로 각각 리뷰들의 단어를 숫자로 변환해준다.
train_sequence = tokenizer.texts_to_sequences(train_review)
train_sequence # 확인

In [ ]:
# 딥러닝 모델의 input size가 고정된 길이를 가진다.

# 리뷰 마다 길이가 다를 것이다.
# 예시
# 이영화너무재밌음 8자
# 이영화다음에가족들이랑또보러올거야 17자

# 딥러닝 input size가 17자리라면 못들어간다.
# 사이즈를 맞추자 -> padding 뭘로 채운다.

# padding 예시
# 이영화너무재밌음 0 0 0 0 0 0 0 0 0 17자
# 이영화다음에가족들이랑또보러올거야 17자

train_input = pad_sequences(train_sequence, maxlen=8, padding="post")

# maxlen=8: 패딩을 할건데 8자리까지 맞춰줄거임
# padding="post": 뒤에서부터 0으로 채울것임
train_input

In [ ]:
# 정답값

train_label = np.array(train['label'])
train_label


In [ ]:
# 모델 만들기

# 데이터를 8(학습):2(평가) 비율로 쪼개기 위한 함수
from sklearn.model_selection import train_test_split

# 학습데이터, 평가데이터, 학습정답, 평가정답      # 피처데이터, 정답데이터, val 데이터 사이즈 비율
x_train, x_val, y_train, y_val = train_test_split(train_input, train_label[:500], test_size=0.2)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense, Embedding

model = Sequential() # 모델 객체 정의
word_size = len(tokenizer.word_index)+1
model.add(Embedding(word_size, 128, input_length = 8)) # 워드사이즈,  128의 아웃풋, 8사이즈의 인풋
model.add(Flatten()) # 위 임베딩 결과가 2차원이라서 flatten을 하여서 1D 벡터로 만듦
model.add(Dense(1,activation='relu')) # activation function 중 하나인 relu를 거쳐서 아웃풋 1로
model.compile(optimizer="adam",loss="binary_crossentropy", metrics =['accuracy'])
             # 모델 설정부분, optimizer는 adam, loss는 binary_crossentropy로 계산,
             # 모델 퍼포먼스는 accuracy로 측정해라.

model.fit(x_train,y_train, epochs=5, batch_size = 32) # 모델 학습을 x_train(데이터값),y_train(정답값)으로 시작해라

In [ ]:
model.evaluate(x_val,y_val)


In [ ]:
text = "이 영화 너무 다시볼거야 너무 재밌다"

re_text = preprocessing(text, okt) # 전처리: 정규표현식, 어간분리, 스톱워드처리
text_data = []
text_data.append(re_text) # n x n의 형태로 만들어줘야함, 데이터가 1개이니까
                          # [[단어리스트]] 이렇게 들어가야함.
                          # 데이터가 2개면
                          # [[단어리스트],[단어리스트]] 이렇게 2 x n으로 들어가줘야함.
text_seq = tokenizer.texts_to_sequences(text_data) # 단어리스트를 숫자리스트로 변환
text_seq = pad_sequences(text_seq, maxlen = 11, padding = "post") # 8사이즈로 패딩처리해야하니까
model.predict(text_seq) # 모델에 넣고 긍부정 평가, 0으로 갈수록 부정, 1로 갈수록 긍정
                        # 현재 500개의 문장만 들어가서 정확도가 낮다.

LSTM 모델 사용하기

In [ ]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense, Embedding
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(Embedding(word_size, 128, input_length = 8)) # 임베딩 레이어 설정
model.add(LSTM(units=128)) # LSTM 모델정의 units는 아웃풋 피처 갯수
model.add(Dense(1,activation="relu")) # Dense는 LSTM의 아웃풋 피처를 받아서 relu를 거쳐 1개의
                                      # 아웃풋을 낸다.
model.compile(optimizer="adam",loss="binary_crossentropy", metrics =['accuracy'])

early = EarlyStopping(monitor = "val_loss" , mode = "min", verbose = 1, patience = 5)


model.fit(x_train,y_train, epochs=100, batch_size = 32, callbacks = [early],
          validation_split = 0.2) # 전체학습 epochs 5회, batch 사이즈는 32

In [ ]:
model.evaluate(x_val,y_val)


In [ ]:
text = "이 영화 너무 다시볼거야 너무 재밌다"

re_text = preprocessing(text, okt) # 전처리: 정규표현식, 어간분리, 스톱워드처리
text_data = []
text_data.append(re_text) # n x n의 형태로 만들어줘야함, 데이터가 1개이니까
                          # [[단어리스트]] 이렇게 들어가야함.
                          # 데이터가 2개면
                          # [[단어리스트],[단어리스트]] 이렇게 2 x n으로 들어가줘야함.
text_seq = tokenizer.texts_to_sequences(text_data) # 단어리스트를 숫자리스트로 변환
text_seq = pad_sequences(text_seq, maxlen = 8, padding = "post") # 8사이즈로 패딩처리해야하니까
model.predict(text_seq) # 모델에 넣고 긍부정 평가, 0으로 갈수록 부정, 1로 갈수록 긍정
                        # 현재 500개의 문장만 들어가서 정확도가 낮다.